In [5]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import nibabel as nib
import subprocess
from datetime import datetime

from mfs_tools.library.distance_stuff import find_wb_command_path
from mfs_tools.library.utility_stuff import compare_mats


save_to = Path("/mnt/cache/pfm_python/")
cifti_bold_path = (
    save_to /
    "sub-ME01_task-rest_concatenated_demeaned_and_regressed_32k_fsLR.dtseries.nii"
)
surface_files = {
    'lh': Path(
        "/mnt/brunodata/open_data/ds005118/derivatives/sub-ME01/fs_LR/fsaverage_LR32k"
        "/ME01.L.midthickness.32k_fs_LR.surf.gii"
    ),
    'rh': Path(
        "/mnt/brunodata/open_data/ds005118/derivatives/sub-ME01/fs_LR/fsaverage_LR32k"
        "/ME01.R.midthickness.32k_fs_LR.surf.gii"
    ),
}
distance_matrix_path = save_to / "dist_complete.npy"

wb_command_path = find_wb_command_path("/usr/local/workbench/2.0.1/bin_linux64/wb_command")
work_dir = None

matlab_base_path = Path("/mnt/cache/pfm_matlab")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
smoothed_images = dict()
kernel_sizes = ["0.85", "1.7", "2.55", ]
for k in kernel_sizes:
    print(datetime.now())
    output_file = str(cifti_bold_path).replace(
        "_and_regressed_32k", f"_regressed_and_smoothed-{k}_32k"
    )
    command_list = [
        str(wb_command_path), '-cifti-smoothing',
        str(cifti_bold_path), k, k, "COLUMN",
        output_file,
        "-left-surface", str(surface_files['lh']),
        "-right-surface", str(surface_files['rh']),
        "-merged-volume"
    ]
    print("Command: " + " ".join(command_list))
    _p = subprocess.run(command_list)
    if _p.returncode != 0:
        print("  failed")
    else:
        print("  success")

    saved_img = nib.Cifti2Image.from_filename(output_file)
    print(saved_img.dataobj.dtype)

    smoothed_images[k] = {
        'img': saved_img,
        'path': Path(output_file),
        'k': k,
    }
    print(datetime.now())


2025-01-12 17:00:17.636601
Command: /opt/workbench/bin_linux64/wb_command -cifti-smoothing /mnt/cache/pfm_python/sub-ME01_task-rest_concatenated_demeaned_and_regressed_32k_fsLR.dtseries.nii 0.85 0.85 COLUMN /mnt/cache/pfm_python/sub-ME01_task-rest_concatenated_demeaned_regressed_and_smoothed-0.85_32k_fsLR.dtseries.nii -left-surface /mnt/brunodata/open_data/ds005118/derivatives/sub-ME01/fs_LR/fsaverage_LR32k/ME01.L.midthickness.32k_fs_LR.surf.gii -right-surface /mnt/brunodata/open_data/ds005118/derivatives/sub-ME01/fs_LR/fsaverage_LR32k/ME01.R.midthickness.32k_fs_LR.surf.gii -merged-volume
  success
float32
2025-01-12 17:01:13.933464
2025-01-12 17:01:13.933490
Command: /opt/workbench/bin_linux64/wb_command -cifti-smoothing /mnt/cache/pfm_python/sub-ME01_task-rest_concatenated_demeaned_and_regressed_32k_fsLR.dtseries.nii 1.7 1.7 COLUMN /mnt/cache/pfm_python/sub-ME01_task-rest_concatenated_demeaned_regressed_and_smoothed-1.7_32k_fsLR.dtseries.nii -left-surface /mnt/brunodata/open_data/ds0

In [9]:
# Are the output BOLD data the same?
# Note the tolerance. There are differences, probably due to float
# encoding or minor regression fitting between matlab and python.
# These differences exist beyond the fifth decimal point, but for
# this, we don't really care about that. If BOLD values match to
# the hundred-thousandth place, that's way more than close enough for us.
for k, img_dict in smoothed_images.items():
    ml_smoothed_img = nib.Cifti2Image.from_filename(
        str(matlab_base_path / img_dict['path'].name)
    )
    compare_mats(img_dict['img'].get_fdata(), ml_smoothed_img.get_fdata(),
                 a_name=f"python {img_dict['k']}-smoothed BOLD",
                 b_name=f"matlab {img_dict['k']}-smoothed BOLD",
                 tolerance=0.00001, verbose=True, preview=True)


  The matrices 'python 0.85-smoothed BOLD' and 'matlab 0.85-smoothed BOLD' are equal, with tolerance of 1e-05.
  Mem before 3,524.8MB; Mem after 3,525.2MB; delta 0.4
  The matrices 'python 1.7-smoothed BOLD' and 'matlab 1.7-smoothed BOLD' are equal, with tolerance of 1e-05.
  Mem before 5,193.9MB; Mem after 5,193.9MB; delta 0.1
  The matrices 'python 2.55-smoothed BOLD' and 'matlab 2.55-smoothed BOLD' are equal, with tolerance of 1e-05.
  Mem before 6,854.2MB; Mem after 6,854.2MB; delta 0.1
